In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

def load_midi_files():
    notes = []
    midi_files = glob.glob("data/**/*.mid", recursive=True)

    if not midi_files:
        print("⚠ No MIDI files found in 'data/' folder.")
        return []

    print(f"Found {len(midi_files)} MIDI files. Starting processing...\n")

    for idx, file in enumerate(midi_files, start=1):
        print(f"[{idx}/{len(midi_files)}] Processing: {file}")

        try:
            midi = converter.parse(file)
            parts = instrument.partitionByInstrument(midi)
            elements = parts.parts[0].recurse() if parts else midi.flat.notes

            for element in elements:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f"⚠ Error processing {file}: {e}")

    with open("data/notes.pkl", "wb") as f:
        pickle.dump(notes, f)

    print(f"\nProcessing complete! Total notes collected: {len(notes)}")
    return notes

def prepare_sequences(notes, seq_length=100):
    pitchnames = sorted(set(notes))
    note_to_int = {note: num for num, note in enumerate(pitchnames)}

    network_input = []
    network_output = []

    for i in range(len(notes) - seq_length):
        seq_in = notes[i:i + seq_length]
        seq_out = notes[i + seq_length]
        network_input.append([note_to_int[n] for n in seq_in])
        network_output.append(note_to_int[seq_out])

    n_patterns = len(network_input)
    input_tensor = np.reshape(network_input, (n_patterns, seq_length, 1)) / float(len(pitchnames))
    output_tensor = np.array(network_output)

    return input_tensor, output_tensor, note_to_int, pitchnames

if __name__ == "__main__":
    notes = load_midi_files()
    print("Preprocessing completed. You can now run train.py")


Found 295 MIDI files. Starting processing...

[1/295] Processing: data\albeniz\alb_esp1.mid
[2/295] Processing: data\albeniz\alb_esp2.mid
[3/295] Processing: data\albeniz\alb_esp3.mid
[4/295] Processing: data\albeniz\alb_esp4.mid
[5/295] Processing: data\albeniz\alb_esp5.mid
[6/295] Processing: data\albeniz\alb_esp6.mid
[7/295] Processing: data\albeniz\alb_se1.mid
[8/295] Processing: data\albeniz\alb_se2.mid
[9/295] Processing: data\albeniz\alb_se3.mid
[10/295] Processing: data\albeniz\alb_se4.mid
[11/295] Processing: data\albeniz\alb_se5.mid
[12/295] Processing: data\albeniz\alb_se6.mid
[13/295] Processing: data\albeniz\alb_se7.mid
[14/295] Processing: data\albeniz\alb_se8.mid
[15/295] Processing: data\bach\bach_846.mid
[16/295] Processing: data\bach\bach_847.mid
[17/295] Processing: data\bach\bach_850.mid
[18/295] Processing: data\balakir\islamei.mid
[19/295] Processing: data\beeth\appass_1.mid
[20/295] Processing: data\beeth\appass_2.mid
[21/295] Processing: data\beeth\appass_3.mid


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pickle
# from preprocess import load_midi_files, prepare_sequences

# LSTM Model
class MusicLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MusicLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Last time-step
        return out

# Prepare data
notes = load_midi_files()
input_data, output_data, note_to_int, pitchnames = prepare_sequences(notes)

# Convert to Torch tensors
X = torch.tensor(input_data, dtype=torch.float32)
y = torch.tensor(output_data, dtype=torch.long)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Model params
input_size = 1
hidden_size = 256
output_size = len(pitchnames)

model = MusicLSTM(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for batch_X, batch_y in loader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")

# Save model + mappings
torch.save(model.state_dict(), "model.pth")
with open("data/mappings.pkl", "wb") as f:
    pickle.dump((note_to_int, pitchnames), f)


Found 295 MIDI files. Starting processing...

[1/295] Processing: data\albeniz\alb_esp1.mid
[2/295] Processing: data\albeniz\alb_esp2.mid
[3/295] Processing: data\albeniz\alb_esp3.mid
[4/295] Processing: data\albeniz\alb_esp4.mid
[5/295] Processing: data\albeniz\alb_esp5.mid
[6/295] Processing: data\albeniz\alb_esp6.mid
[7/295] Processing: data\albeniz\alb_se1.mid
[8/295] Processing: data\albeniz\alb_se2.mid
[9/295] Processing: data\albeniz\alb_se3.mid
[10/295] Processing: data\albeniz\alb_se4.mid
[11/295] Processing: data\albeniz\alb_se5.mid
[12/295] Processing: data\albeniz\alb_se6.mid
[13/295] Processing: data\albeniz\alb_se7.mid
[14/295] Processing: data\albeniz\alb_se8.mid
[15/295] Processing: data\bach\bach_846.mid
[16/295] Processing: data\bach\bach_847.mid
[17/295] Processing: data\bach\bach_850.mid
[18/295] Processing: data\balakir\islamei.mid
[19/295] Processing: data\beeth\appass_1.mid
[20/295] Processing: data\beeth\appass_2.mid
[21/295] Processing: data\beeth\appass_3.mid


In [3]:
import torch
import numpy as np
import pickle
from music21 import instrument, note, stream, chord
# from train import MusicLSTM

# Load mappings
with open("data/mappings.pkl", "rb") as f:
    note_to_int, pitchnames = pickle.load(f)

int_to_note = {num: note for note, num in note_to_int.items()}

# Prepare model
model = MusicLSTM(1, 256, len(pitchnames))
model.load_state_dict(torch.load("model.pth"))
model.eval()

# Generate sequence
sequence_length = 100
start = np.random.randint(0, len(pitchnames)-sequence_length)
pattern = [note_to_int[note] for note in pitchnames[start:start+sequence_length]]

prediction_output = []

for _ in range(200):  # generate 200 notes
    input_seq = torch.tensor(np.reshape(pattern, (1, sequence_length, 1)) / float(len(pitchnames)), dtype=torch.float32)
    prediction = model(input_seq).detach().numpy()
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:]

# Convert to MIDI
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = [note.Note(int(n)) for n in notes_in_chord]
        for n in notes:
            n.storedInstrument = instrument.Piano()
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='generated_music.mid')
print("Music generated: generated_music.mid")


Music generated: generated_music.mid


In [ ]:
import subprocess
import tkinter as tk
import pygame
import os
from tkinter import filedialog

# Initialize pygame mixer once at the beginning
pygame.mixer.init()

# Configuration paths
fluidsynth_exe = r"C:\Users\mS\Documents\CodeAlpha\Artificial Intelligence\CodeAlpha_AI_tasks\CodeAlpha_Music Generation with AI\fluidsynth-2.4.7-win10-x64\bin\fluidsynth.exe"
sf2_path = r"C:\Users\mS\Documents\CodeAlpha\Artificial Intelligence\CodeAlpha_AI_tasks\CodeAlpha_Music Generation with AI\FluidR3_GM\FluidR3_GM.sf2"
midi_file = "generated_music.mid"
wav_file = "output.wav"

def convert_midi_to_wav():
    """
    Converts MIDI to WAV file without playing it.
    Only performs the conversion and updates status.
    """
    # Stop any currently playing audio first
    pygame.mixer.music.stop()
    pygame.mixer.quit()
    pygame.mixer.init()
    
    if not os.path.exists(midi_file):
        status_label.config(text="Error: MIDI file not found!", fg="red")
        return

    # Ensure no audio is playing during conversion
    status_label.config(text="Converting... Please wait", fg="orange")
    root.update()  # Force UI update to show conversion message

    try:
        # Run conversion with quiet flag to prevent automatic playback
        subprocess.run([
            fluidsynth_exe,
            "-ni",  # Non-interactive mode
            "-q",   # Quiet mode (suppresses output)
            sf2_path,
            midi_file,
            "-F", wav_file,
            "-r", "44100"
        ], check=True)

        if os.path.exists(wav_file):
            status_label.config(text="Conversion complete!\nClick Play to listen", fg="green")
        else:
            status_label.config(text="Error: Conversion failed - no output file", fg="red")
            
    except subprocess.CalledProcessError as e:
        status_label.config(text=f"Conversion error: {e}", fg="red")
    except Exception as e:
        status_label.config(text=f"Unexpected error: {e}", fg="red")

def play_wav():
    """Plays the converted WAV file only if it exists"""
    if not os.path.exists(wav_file):
        status_label.config(text="Error: No WAV file found!\nConvert first", fg="red")
        return
    
    try:
        pygame.mixer.music.load(wav_file)
        pygame.mixer.music.play()
        status_label.config(text="Now playing...", fg="blue")
    except pygame.error as e:
        status_label.config(text=f"Playback error: {e}", fg="red")

def save_wav():
    """Saves the WAV file to user-selected location"""
    if not os.path.exists(wav_file):
        status_label.config(text="Error: No WAV file found!\nConvert first", fg="red")
        return
    
    try:
        save_path = filedialog.asksaveasfilename(
            defaultextension=".wav",
            filetypes=[("WAV files", "*.wav")],
            title="Save WAV File As",
            initialfile="converted_music.wav"
        )
        
        if save_path:
            import shutil
            shutil.copyfile(wav_file, save_path)
            status_label.config(text=f"Saved successfully to:\n{save_path}", fg="green")
    except Exception as e:
        status_label.config(text=f"Save error: {e}", fg="red")

# GUI Setup
root = tk.Tk()
root.title("MIDI to WAV Converter")
root.geometry("400x350")
root.resizable(False, False)
root.configure(bg="#f0f0f0")

# Main frame
main_frame = tk.Frame(root, bg="#f0f0f0")
main_frame.pack(pady=20)

# Buttons
btn_convert = tk.Button(
    main_frame,
    text="Convert MIDI to WAV",
    command=convert_midi_to_wav,
    bg="#4CAF50",
    fg="white",
    font=("Arial", 12, "bold"),
    width=20
)
btn_convert.pack(pady=8)

btn_play = tk.Button(
    main_frame,
    text="Play WAV",
    command=play_wav,
    bg="#2196F3",
    fg="white",
    font=("Arial", 12, "bold"),
    width=20
)
btn_play.pack(pady=8)

btn_save = tk.Button(
    main_frame,
    text="Save WAV File",
    command=save_wav,
    bg="#FF9800",
    fg="white",
    font=("Arial", 12, "bold"),
    width=20
)
btn_save.pack(pady=8)

# Status label
status_label = tk.Label(
    root,
    text="Ready to convert MIDI to WAV",
    bg="#f0f0f0",
    fg="#333333",
    font=("Arial", 11),
    wraplength=350,
    justify="center"
)
status_label.pack(pady=15)

root.mainloop()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import subprocess
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pygame
import os
import time

# Initialize pygame mixer
pygame.mixer.init()

# Paths
fluidsynth_exe = r"C:\Users\mS\Documents\CodeAlpha\Artificial Intelligence\CodeAlpha_AI_tasks\CodeAlpha_Music Generation with AI\fluidsynth-2.4.7-win10-x64\bin\fluidsynth.exe"
sf2_path = r"C:\Users\mS\Documents\CodeAlpha\Artificial Intelligence\CodeAlpha_AI_tasks\CodeAlpha_Music Generation with AI\FluidR3_GM\FluidR3_GM.sf2"
midi_file = "generated_music.mid"
wav_file = "output.wav"

class MidiConverterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Professional MIDI to WAV Converter")
        self.root.geometry("450x300")
        self.root.configure(bg="#f0f0f0")
        self.root.resizable(False, False)
        
        # Styles
        self.style = ttk.Style()
        self.style.configure('TButton', font=('Arial', 10), padding=6)
        self.style.configure('TLabel', background="#f0f0f0", font=('Arial', 10))
        
        # Main Frame
        self.main_frame = ttk.Frame(root, padding="20", style='TFrame')
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Title
        self.title_label = ttk.Label(
            self.main_frame,
            text="MIDI to WAV Converter",
            font=('Arial', 14, 'bold')
        )
        self.title_label.grid(row=0, column=0, columnspan=2, pady=10)
        
        # Buttons Frame
        self.btn_frame = ttk.Frame(self.main_frame)
        self.btn_frame.grid(row=1, column=0, columnspan=2, pady=15)
        
        # Convert Button
        self.convert_btn = ttk.Button(
            self.btn_frame,
            text="Convert MIDI to WAV",
            command=self.convert_midi_to_wav,
            style='TButton'
        )
        self.convert_btn.pack(side=tk.LEFT, padx=10)
        
        # Play Button
        self.play_btn = ttk.Button(
            self.btn_frame,
            text="Play Music",
            command=self.play_wav,
            style='TButton'
        )
        self.play_btn.pack(side=tk.LEFT, padx=10)
        
        # Save Button
        self.save_btn = ttk.Button(
            self.btn_frame,
            text="Save WAV",
            command=self.save_wav,
            style='TButton'
        )
        self.save_btn.pack(side=tk.LEFT, padx=10)
        
        # Status Frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.grid(row=2, column=0, columnspan=2, pady=10)
        
        # Status Label
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        self.status_label = ttk.Label(
            self.status_frame,
            textvariable=self.status_var,
            font=('Arial', 10, 'italic')
        )
        self.status_label.pack()
        
        # Progress Bar
        self.progress = ttk.Progressbar(
            self.main_frame,
            orient=tk.HORIZONTAL,
            length=300,
            mode='determinate'
        )
        self.progress.grid(row=3, column=0, columnspan=2, pady=10)
        
        # Info Label
        self.info_label = ttk.Label(
            self.main_frame,
            text=f"MIDI: {midi_file} → WAV: {wav_file}",
            font=('Arial', 8),
            foreground='gray'
        )
        self.info_label.grid(row=4, column=0, columnspan=2, pady=5)
        
    def convert_midi_to_wav(self):
        """Convert MIDI to WAV with progress indication"""
        if not os.path.exists(midi_file):
            self.update_status("Error: MIDI file not found!", "red")
            messagebox.showerror("Error", "MIDI file not found!")
            return
            
        self.update_status("Converting... Please wait", "orange")
        self.progress['value'] = 0
        self.root.update()
        
        try:
            # Simulate progress (since actual conversion is fast)
            for i in range(101):
                self.progress['value'] = i
                self.root.update()
                time.sleep(0.02)
            
            subprocess.run([
                fluidsynth_exe,
                "-ni",
                "-q",
                sf2_path,
                midi_file,
                "-F", wav_file,
                "-r", "44100"
            ], check=True)
            
            self.progress['value'] = 100
            self.update_status("Conversion complete! Ready to play", "green")
            messagebox.showinfo("Success", "MIDI converted to WAV successfully!")
            
        except Exception as e:
            self.update_status(f"Error: {str(e)}", "red")
            messagebox.showerror("Error", f"Conversion failed: {str(e)}")
    
    def play_wav(self):
        """Play WAV file with status updates"""
        if not os.path.exists(wav_file):
            self.update_status("No WAV file found! Convert first.", "red")
            messagebox.showwarning("Warning", "No WAV file found. Please convert first.")
            return
            
        try:
            pygame.mixer.music.stop()
            pygame.mixer.quit()
            pygame.mixer.init()
            
            self.update_status("Playing...", "blue")
            pygame.mixer.music.load(wav_file)
            pygame.mixer.music.play()
            
            # Show duration message
            duration = pygame.mixer.Sound(wav_file).get_length()
            self.root.after(int(duration * 1000), lambda: 
                self.update_status("Playback finished", "green"))
                
        except pygame.error as e:
            self.update_status(f"Playback error: {str(e)}", "red")
            messagebox.showerror("Error", f"Playback failed: {str(e)}")
    
    def save_wav(self):
        """Save WAV file to selected location"""
        if not os.path.exists(wav_file):
            self.update_status("No WAV file to save", "red")
            messagebox.showwarning("Warning", "No WAV file found. Please convert first.")
            return
            
        save_path = filedialog.asksaveasfilename(
            defaultextension=".wav",
            filetypes=[("WAV files", "*.wav"), ("All files", "*.*")],
            title="Save WAV File As"
        )
        
        if save_path:
            try:
                import shutil
                shutil.copy2(wav_file, save_path)
                self.update_status(f"Saved to {os.path.basename(save_path)}", "green")
                messagebox.showinfo("Success", "File saved successfully!")
            except Exception as e:
                self.update_status(f"Save error: {str(e)}", "red")
                messagebox.showerror("Error", f"Failed to save: {str(e)}")
    
    def update_status(self, message, color):
        """Update status label with color coding"""
        self.status_var.set(message)
        if color == "red":
            self.status_label.config(foreground="red")
        elif color == "green":
            self.status_label.config(foreground="green")
        elif color == "blue":
            self.status_label.config(foreground="blue")
        elif color == "orange":
            self.status_label.config(foreground="orange")
        else:
            self.status_label.config(foreground="black")

if __name__ == "__main__":
    root = tk.Tk()
    app = MidiConverterApp(root)
    root.mainloop()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
